In [45]:
## 
import pandas as pd
import numpy as np
import os

"""
DEFINE PROGRAM CONSTANTS
"""
# This is the file that is being read in
inputFile = r"Domestic X and M prices.xlsx"

# These are the two files that will be created / updated
outFile1 = "XFromProv.csv"
outFile2 = "MToProv.csv"

# Start by assigning the current working directory
currentDir = 'D:/Robert Docs/College/NIU/PhD/SP18_Research'

# A list of all of the provinces we are interested in, only data from these provinces will be recorded
#  in the output tables, anything else will be ignored.
ProvinceList = ["Alagoas", "Amazonas", "Bahia", "Ceara", "Espirito Santo", "Goias", "Maranhao", "Mato Grosso", "Minas Gerais",
                "Para", "Paraiba", "Parana", "Pernambuco", "Piaui", "Rio De Janeiro", "Rio Grande do Norte", "Rio Grande do Sul",
				"Santa Catarina", "Sao Paulo", "Sergipe"]

# targetCol defines the column in each sub-block we're looking for, this can be modified in the future to map other
#  quantities of interest. This is an integer totalizer field, so it counts how many columns over in each sub block to use
targetName = ["value per unit", "price per unit", "avg prices reported per year"]

# identifier defines what connects two rows together, signifying the beginning of a sub-block
identifier = "source link"

"""
BEGIN CODE
"""

print("Attempting to read input file from " + currentDir)

xlsx = pd.ExcelFile(currentDir + '/' + inputFile)
# Ref: Sheet 0: X from Prov to other provs, Note: As the first row read is considered a header, we only skip the first
df1 = xlsx.parse(1, skiprows=0, header=None)
# Ref: Sheet 1: M to Prov from other provs
df2 = xlsx.parse(2, skiprows=0, header=None)

print("Parsing sheet 1")
# Define Sheet 1 locals
targetCol = 0
saveDict = { }
sheet1 = []
colIndex = 0
subBlocks = []
geoSpatial = []
for i, row in enumerate(df1.itertuples(), 0):
    sheet1.append(tuple((i, row)))

for (i, row) in sheet1:   
    # Our first step will be to construct the "sub-blocks", so check for any "named" column, and then below it for the 
    #  identifier
    rowSeries = pd.Series(row)
    # The first two columns contain the geo-temporal information, but we need to make sure it's not "subheaders"
    #  This is a simple digit test on the first four characters in the string of the "year" value
    if(str(rowSeries[1])[0:4].isdigit()):
        # Grab the second column as well
        val = (str(rowSeries[1]).strip())[0:4]
        # We now need to define the starting point of this block so the program can identify the link correctly.
        #  To do this, we loop back up repeatidly until we find the block that has a string or is empty.
        foundStart = False
        currentIndex = i
        linkRow = 0
        while(foundStart == False):
            prev = str(pd.Series(sheet1[currentIndex])[1][1])[0:4]
            if(prev.isdigit() == False):
                # Got it!
                linkRow = currentIndex + 1
                break
            # Keep going...
            currentIndex -= 1
            if(currentIndex < 0):
                print("ERROR: Cannot locate link")
                break
        append = tuple((int(i), int(linkRow), val, rowSeries[2]))
        geoSpatial.append(append)
        
    for col in rowSeries:
        # Check for any open blocks with text and nans in both the spots to it's left and right
        if(colIndex >= 1 and colIndex < rowSeries.size-1):
            test = col
            testPrev = rowSeries[colIndex - 1]
            testNext = rowSeries[colIndex + 1]
            if(testPrev == "Destination if known"):
                testPrev = np.nan
            if((pd.isnull(test) == False) and (pd.isnull(testPrev) and pd.isnull(testNext)) and (i < len(sheet1)-1)):
                # It might be a column header we're looking for, peek at the next row and check for the link
                nextRow = sheet1[i+1][1]
                testCompare = str(nextRow[colIndex]).strip().lower()
                if((testCompare == identifier) or (testCompare[0:5] == "http:") or (testCompare[0:6] == "https:")):
                    # We have a match! Now find the column we're looking for
                    for ix in range(colIndex, colIndex+4):
                        if(ix < rowSeries.size - 1):
                            for indTarget in targetName:
                                if(indTarget in str(df1.iloc[:, ix]).lower()):
                                    targetCol = ix+1
                                    break
                    if(str(sheet1[i+2][1][targetCol]).isdigit()):
                        subBlocks.append(tuple((i+2, col, targetCol)))
                    else:
                        subBlocks.append(tuple((i+3, col, targetCol)))
                    targetCol = 0
        colIndex += 1
    colIndex = 0
    
# Now, reconstruct the new table, we'll start by assembling a full list of all of the headers we want
stored_header_info = []
headers = ['Province']
for subBlock in subBlocks:
    for geoTemp in geoSpatial:
        if(geoTemp[1] == subBlock[0]):
            # These blocks are linked, grab the product name and year
            fixName = subBlock[1].strip()
            fixName = fixName.replace(" ", "-")
            fixName = fixName.replace(",", "")
            fixName = fixName.replace(" ", "")
            hName = fixName + "_" + geoTemp[2]
            # Store the header information for the table writing
            if(not (hName in headers)):
                headers.append(hName)
                stored_header_info.append(tuple((fixName, geoTemp[0], subBlock[2], geoTemp[3], geoTemp[2])))
            # Capture the value we want for saving and place it in a dictionary
            vStr = str(sheet1[geoTemp[0]][1][subBlock[2]])
            vStr = vStr.replace("$", ",")
            dKey = (hName + '_' + geoTemp[3].replace(" ", "_")).lower()
            if(vStr):
                saveDict[dKey] = vStr                
            print("Saving: " + dKey + " => " + vStr)
outTable1 = pd.DataFrame(columns=headers)

print("Writing New Table")
# Next, we're going to loop through each Province in our list and grab any connected data points
tmp = []
for province in ProvinceList:
    tmp.append(province)
    for header in stored_header_info:
        keyTest = (header[0] + "_" + header[4] + "_" + province.replace(" ", "_")).lower()
        if(keyTest in saveDict):
            tmp.append(saveDict[keyTest])        
        else:
            tmp.append(np.nan)
    sObj = pd.Series(tmp, index=headers)
    outTable1 = outTable1.append(sObj, ignore_index=True)
    tmp = []
# Write the output
outTable1.to_csv(outFile1)

Attempting to read input file from D:/Robert Docs/College/NIU/PhD/SP18_Research
Parsing sheet 1
Saving: aguardente_1860_alagoas => 697.749065894
Saving: aguardente_1861_alagoas => 413.978206023
Saving: aguardente_1862_alagoas => 387.642276423
Saving: cotton_1860_alagoas => 6618.43404544
Saving: cotton_1861_alagoas => 8284.84700828
Saving: cotton_1862_alagoas => 17460.1133445
Saving: sugar_1860_alagoas => 2469.35140994
Saving: sugar_1861_alagoas => 2213.93581515
Saving: sugar_1862_alagoas => 2444.41207289
Saving: cooking-oil_1860_alagoas => nan
Saving: cooking-oil_1861_alagoas => 1098.31438652
Saving: cooking-oil_1862_alagoas => 1451.49700599
Saving: rice_1860_alagoas => 3462.99212598
Saving: rice_1861_alagoas => 1710.16393443
Saving: rice_1862_alagoas => nan
Saving: crackers_1860_alagoas => nan
Saving: crackers_1861_alagoas => nan
Saving: crackers_1862_alagoas => 20000
Saving: potatoes_1860_alagoas => nan
Saving: potatoes_1861_alagoas => 160000
Saving: potatoes_1862_alagoas => 4228.346